In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from tqdm import tqdm_notebook
# Census API Key
from config import api_key
c = Census(api_key, year=2017)

In [2]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",'B15003_002E', 'B15003_003E', 
                          'B15003_004E', 'B15003_005E', 'B15003_006E', 'B15003_007E', 'B15003_008E',
                          'B15003_009E', 'B15003_010E', 'B15003_011E', 'B15003_012E', 'B15003_013E',
                          'B15003_014E', 'B15003_015E', 'B15003_016E', 'B15003_017E', 'B15003_018E',
                          'B15003_019E', 'B15003_020E', "B15003_021E", "B15003_022E", "B15003_023E",
                          "B15003_024E", "B15003_025E", 'B15003_001E', 'B12001_001E', 'B12001_003E',
                          'B12001_004E', 'B12001_007E', 'B12001_009E', 'B12001_010E', 'B12001_011E', 
                          'B12001_012E', 'B12001_013E', 'B12001_016E', 'B12001_018E', 'B12001_019E')
                          , {'for': 'zip code tabulation area:*'})


# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Column Reordering
census_df = census_df.rename(columns={'B01003_001E': 'Population',
                                      'B01002_001E': 'Median Age',
                                      'B19013_001E': 'Household Income',
                                      'B19301_001E': 'Per Capita Income',
                                      'B17001_002E': 'Poverty Count',
                                      'NAME': 'Name', 
                                      'zip code tabulation area': 'Zip Code',
                                      'B15003_021E' : 'Associate\'s Degree Count',
                                      'B15003_022E' : 'Bachelor\'s Degree Count',
                                      'B15003_023E' : 'Master\'s Degree Count',
                                      'B15003_024E' : 'Professional School Degree Count',
                                      'B15003_025E' : 'Doctorate Degree Count',
                                      'B15003_001E' : 'Total educational attainment over 25',
                                      'B15003_002E' : 'No School',
                                      'B15003_003E' : 'Nursery school',
                                      'B15003_004E' : 'Kindergarten',
                                      'B15003_005E' : '1st grade',
                                      'B15003_006E' : '2nd grade',
                                      'B15003_007E' : '3rd grade',
                                      'B15003_008E' : '4th grade',
                                      'B15003_009E' : '5th grade',
                                      'B15003_010E' : '6th grade',
                                      'B15003_011E' : '7th grade',
                                      'B15003_012E' : '8th grade',
                                      'B15003_013E' : '9th grade',
                                      'B15003_014E' : '10th grade',
                                      'B15003_015E' : '11th grade',
                                      'B15003_016E' : '12th grade No Diploma',
                                      'B15003_017E' : 'HS Diploma',
                                      'B15003_018E' : 'GED',
                                      'B15003_019E' : 'Some College less than 1 year',
                                      'B15003_020E' : 'Some College More than 1 year',
                                      'B12001_001E' : 'Over 15 Marital Status',
                                      'B12001_003E' : 'Male Never Married',
                                      'B12001_004E' : 'Male Married',
                                      'B12001_007E' : 'Male Spouse Seperated',
                                      'B12001_009E' : 'Male Widowed',
                                      'B12001_010E' : 'Male Divorced',
                                      'B12001_012E' : 'Female Never Married',
                                      'B12001_013E' : 'Female Married',
                                      #'B12001_014E' : 'Female Spouse present',
                                      #'B12001_015E' : 'Female Spouse absent',
                                      'B12001_016E' : 'Female Spouse Seperated',
                                      #'B12001_017E' : 'Female Spouse Other',
                                      'B12001_018E' : 'Female Widowed',
                                      'B12001_019E' : 'Female Divorced'})

census_df["Poverty Rate"] = (100 * census_df["Poverty Count"].astype(int)
                            / census_df["Population"].astype(int))
census_df["Total College Degrees"] = (census_df['Associate\'s Degree Count'].astype(int)+
                             census_df['Bachelor\'s Degree Count'].astype(int)+
                             census_df['Master\'s Degree Count'].astype(int)+
                             census_df['Professional School Degree Count'].astype(int)+
                             census_df['Doctorate Degree Count'].astype(int))
census_df['Total No HS Diploma'] = (census_df['No School'].astype(int)+
                                    census_df['Nursery school'].astype(int)+
                                    census_df['Kindergarten'].astype(int)+
                                    census_df['1st grade'].astype(int)+
                                    census_df['2nd grade'].astype(int)+
                                    census_df['3rd grade'].astype(int)+
                                    census_df['4th grade'].astype(int)+
                                    census_df['5th grade'].astype(int)+
                                    census_df['6th grade'].astype(int)+
                                    census_df['7th grade'].astype(int)+
                                    census_df['8th grade'].astype(int)+
                                    census_df['9th grade'].astype(int)+
                                    census_df['10th grade'].astype(int)+
                                    census_df['11th grade'].astype(int)+
                                    census_df['12th grade No Diploma'].astype(int))
census_df['Total HS Diploma'] = (census_df['HS Diploma'].astype(int)+
                                    census_df['GED'].astype(int))

census_df['Total Some College'] = (census_df['Some College less than 1 year'].astype(int)+
                                    census_df['Some College More than 1 year'].astype(int))

census_df['Education Check'] = (census_df['Total College Degrees'].astype(int)+
                             census_df['Total No HS Diploma'].astype(int)+
                             census_df['Total HS Diploma'].astype(int)+
                             census_df['Total Some College'].astype(int))

census_df["Never Married"] = (census_df['Male Never Married'].astype(int)+
                             census_df['Female Never Married'].astype(int))


census_df["Married"] = (census_df['Male Married'].astype(int)+
                        census_df['Female Married'].astype(int))
                        
census_df["Widowed"] = (census_df['Male Widowed'].astype(int)+
                             census_df['Female Widowed'].astype(int))

census_df["Divorced"] = (census_df['Male Divorced'].astype(int)+
                             census_df['Female Divorced'].astype(int))

census_df['Married Check'] = (census_df['Married'].astype(int)+
                             census_df['Widowed'].astype(int)+
                             census_df['Divorced'].astype(int)+
                             census_df["Never Married"].astype(int))

census_df['College Degree Rate'] = (census_df['Total College Degrees']/census_df['Total educational attainment over 25']*100)

census_df['No High School Diploma Rate'] = (census_df['Total No HS Diploma']/census_df['Total educational attainment over 25']*100)

census_df['HS Diploma Rate'] = (census_df['Total HS Diploma']/census_df['Total educational attainment over 25']*100)

census_df['Some College Rate'] = (census_df['Total Some College']/census_df['Total educational attainment over 25']*100)

census_df['Married Rate'] = (census_df['Married']/census_df['Over 15 Marital Status']*100)

census_df['Widowed Rate'] = (census_df['Widowed']/census_df['Over 15 Marital Status']*100)

census_df['Divorced Rate'] = (census_df['Divorced']/census_df['Over 15 Marital Status']*100)

census_df['Never Married Rate'] = (census_df['Never Married']/census_df['Over 15 Marital Status']*100)

# Final DataFrame
census_df = census_df[['Zip Code', 'Population', 'Median Age', 'Household Income',
                       'Per Capita Income', 'Poverty Count', 'Poverty Rate', 'Total No HS Diploma', 'No High School Diploma Rate',
                       'Total HS Diploma', 'HS Diploma Rate', 'Total Some College', 'Some College Rate', 'Total College Degrees',
                       'College Degree Rate', 'Total educational attainment over 25','Never Married', 'Never Married Rate', 'Married','Married Rate', 'Widowed',
                       'Widowed Rate', 'Divorced', 'Divorced Rate', 'Over 15 Marital Status']]

# Visualize
print(len(census_df))

33120


In [3]:
census_df.loc[census_df['Household Income'] < 0, 'Household Income'] = 0
census_df.loc[census_df['Median Age'] < 0, 'Median Age'] = 0
census_df.loc[census_df['Per Capita Income'] < 0, 'Per Capita Income'] = 0

In [4]:
census_df=census_df.sort_values(by='Zip Code', ascending=True)
census_df.reset_index(drop=True)
census_df.head()

,Zip Code,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Total No HS Diploma,No High School Diploma Rate,Total HS Diploma,...,Total educational attainment over 25,Never Married,Never Married Rate,Married,Married Rate,Widowed,Widowed Rate,Divorced,Divorced Rate,Over 15 Marital Status
0,00601,17599.0,38.9,11757.0,7041.0,11282.0,64.105915,4288,36.127728,2967,...,11869.0,5629,39.262049,5846,40.775616,863,6.019390,1999,13.942945,14337.0
1,00602,39209.0,40.9,16190.0,8978.0,20428.0,52.100283,9407,34.176203,6322,...,27525.0,12079,36.623006,14586,44.224122,2136,6.476260,4181,12.676611,32982.0
2,00603,50135.0,40.4,16645.0,10897.0,25176.0,50.216416,10310,29.464719,9657,...,34991.0,14971,35.963774,17614,42.312866,3098,7.442106,5945,14.281253,41628.0
3,00606,6304.0,42.8,13387.0,5960.0,4092.0,64.911168,2147,49.401749,1351,...,4346.0,2423,45.994685,1880,35.687168,431,8.181473,534,10.136674,5268.0
4,00610,27590.0,41.4,18741.0,9266.0,12553.0,45.498369,6260,32.553302,6032,...,19230.0,8081,34.993288,10581,45.819079,1772,7.673321,2659,11.514312,23093.0


In [5]:
census_df.describe()

,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Total No HS Diploma,No High School Diploma Rate,Total HS Diploma,HS Diploma Rate,...,Total educational attainment over 25,Never Married,Never Married Rate,Married,Married Rate,Widowed,Widowed Rate,Divorced,Divorced Rate,Over 15 Marital Status
count,33120.000000,33120.000000,33120.000000,32795.000000,33120.000000,32803.000000,33120.000000,32762.000000,33120.000000,32762.000000,...,33120.000000,33120.000000,32802.000000,33120.000000,32802.000000,33120.000000,32802.000000,33120.000000,32802.000000,33120.000000
mean,9796.435085,41.619813,53057.399004,28485.554780,1424.893327,14.272287,846.658967,12.831084,1804.061866,33.606431,...,6601.900272,2633.141274,27.930627,3975.534964,54.006754,464.165066,6.757100,863.755223,11.305519,7936.596528
std,14510.547644,10.375402,27263.947714,13536.513115,2760.345208,11.432327,1770.216180,10.285843,2562.183128,13.202915,...,9639.005882,4453.160777,13.210617,5758.359359,13.804021,674.138670,5.443976,1271.301267,6.691947,11643.895459
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,707.000000,36.500000,39005.000000,21159.500000,76.000000,6.687964,48.000000,5.987075,175.000000,25.738249,...,488.000000,135.000000,20.790992,319.000000,47.868240,35.000000,4.368597,59.000000,8.121678,585.000000
50%,2804.000000,41.700000,50635.500000,26122.000000,327.000000,11.777184,210.000000,10.520269,661.000000,33.939723,...,1933.500000,583.000000,25.808879,1246.000000,55.555556,147.000000,6.145542,247.000000,10.843547,2299.000000
75%,13290.250000,47.000000,64560.750000,32639.000000,1462.000000,18.908204,858.000000,17.137468,2412.000000,41.347702,...,9023.250000,3155.250000,32.687442,5413.500000,61.878087,642.000000,8.175332,1167.000000,13.633780,10808.500000
max,119204.000000,89.500000,250001.000000,298129.000000,38839.000000,100.000000,36261.000000,100.000000,28207.000000,100.000000,...,73082.000000,44225.000000,100.000000,52149.000000,100.000000,6578.000000,100.000000,10926.000000,100.000000,88221.000000


In [6]:
zipcode_geo = "uszips.csv"
zipcode_geo_df = pd.read_csv(zipcode_geo, dtype=str, low_memory=False)

In [7]:
zipcode_geo_df = zipcode_geo_df[['zip', 'lat', 'lng', 'city', 'state_id', 'state_name']]
zipcode_geo_df = zipcode_geo_df.rename(columns={'zip': 'Zip Code'})

In [8]:
zipcode_geo_df.head()

,Zip Code,lat,lng,city,state_id,state_name
0,00601,18.18004,-66.75218,Adjuntas,PR,Puerto Rico
1,00602,18.36073,-67.17517,Aguada,PR,Puerto Rico
2,00603,18.45439,-67.12202,Aguadilla,PR,Puerto Rico
3,00606,18.16724,-66.93828,Maricao,PR,Puerto Rico
4,00610,18.29032,-67.12243,Anasco,PR,Puerto Rico


In [9]:
Project_census_df = census_df.merge(zipcode_geo_df, on="Zip Code", how='right')
#census_df = census_df.sort_values(by='Zip Code', ascending=True)
Project_census_df.head()

,Zip Code,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Total No HS Diploma,No High School Diploma Rate,Total HS Diploma,...,Widowed,Widowed Rate,Divorced,Divorced Rate,Over 15 Marital Status,lat,lng,city,state_id,state_name
0,00601,17599.0,38.9,11757.0,7041.0,11282.0,64.105915,4288,36.127728,2967,...,863,6.019390,1999,13.942945,14337.0,18.18004,-66.75218,Adjuntas,PR,Puerto Rico
1,00602,39209.0,40.9,16190.0,8978.0,20428.0,52.100283,9407,34.176203,6322,...,2136,6.476260,4181,12.676611,32982.0,18.36073,-67.17517,Aguada,PR,Puerto Rico
2,00603,50135.0,40.4,16645.0,10897.0,25176.0,50.216416,10310,29.464719,9657,...,3098,7.442106,5945,14.281253,41628.0,18.45439,-67.12202,Aguadilla,PR,Puerto Rico
3,00606,6304.0,42.8,13387.0,5960.0,4092.0,64.911168,2147,49.401749,1351,...,431,8.181473,534,10.136674,5268.0,18.16724,-66.93828,Maricao,PR,Puerto Rico
4,00610,27590.0,41.4,18741.0,9266.0,12553.0,45.498369,6260,32.553302,6032,...,1772,7.673321,2659,11.514312,23093.0,18.29032,-67.12243,Anasco,PR,Puerto Rico


In [10]:
Project_census_df = Project_census_df.loc[Project_census_df['state_name'] != 'Puerto Rico']

In [11]:
Project_census_df.loc[Project_census_df['state_name'] == 'Puerto Rico']

,Zip Code,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Total No HS Diploma,No High School Diploma Rate,Total HS Diploma,...,Widowed,Widowed Rate,Divorced,Divorced Rate,Over 15 Marital Status,lat,lng,city,state_id,state_name


In [12]:
Project_census_df.to_csv('Project1_Zipcode_Data.csv',encoding="utf-8", index=False)